# V2 Model Server (SKLearn)

In [1]:
import mlrun

> 2020-10-19 13:31:31,010 [warning] Failed resolving version info. Ignoring and using defaults


### Configuration and package dependencies

In [14]:
%nuclio config kind="serving"
%nuclio config spec.build.baseImage="mlrun/ml-models"
%nuclio config spec.maxReplicas = 1

%nuclio: setting kind to 'serving'
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'
%nuclio: setting spec.maxReplicas to 1


### Serving class code

In [4]:
from cloudpickle import load
from typing import List
from sklearn.datasets import load_iris
import numpy as np
import mlrun

In [5]:
class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body['inputs'])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [6]:
# nuclio: end-code

# Convert to function object

### Document and save (as a template)

In [15]:
fn = mlrun.code_to_function('v2-model-server', 
                            description="generic sklearn model server",
                            categories=['serving', 'ml'],
                            labels={'author': 'yaronh', 'framework': 'sklearn'},
                            code_output='.')

fn.spec.default_class = 'ClassifierModel'
#print(fn.to_yaml())
fn.export()

> 2020-10-19 13:33:27,973 [info] function spec saved to path: function.yaml


### Configure and add model(s)

In [16]:
models_path = 'https://s3.wasabisys.com/iguazio/models/iris/model.pkl'
fn.add_model('mymodel', model_path=models_path)
#fn.verbose = True

# Test models locally (using a server emulator)

In [17]:
# create an emulator (mock server) from the function configuration)
server = fn.to_mock_server(globals())

{'routes': <mlrun.model.ObjectDict object at 0x7fce176cb810>}
{'model_path': 'https://s3.wasabisys.com/iguazio/models/iris/model.pkl'}
> 2020-10-19 13:33:29,370 [info] model mymodel was loaded
> 2020-10-19 13:33:29,370 [info] model mymodel was loaded
> 2020-10-19 13:33:29,371 [info] Loaded ['mymodel']
> 2020-10-19 13:33:29,371 [info] Loaded ['mymodel']


### Test against the iris dataset 

In [18]:
iris = load_iris()
x = iris['data'].tolist()

In [19]:
result = server.test("/v2/models/mymodel/infer", {"inputs": x})
result.keys()

> 2020-10-19 13:33:29,390 [debug] router run model mymodel, op=infer
> 2020-10-19 13:33:29,390 [debug] router run model mymodel, op=infer


dict_keys(['id', 'model_name', 'outputs'])

##  Deploy server

In [20]:
fn.apply(mlrun.mount_v3io())

In [23]:
fn.deploy(project='v2-srv')

> 2020-10-19 13:43:26,111 [info] deploy started
[nuclio] 2020-10-19 13:43:28,219 (info) Build complete
[nuclio] 2020-10-19 13:43:36,312 (info) Function deploy complete
[nuclio] 2020-10-19 13:43:36,322 done updating v2-srv-v2-model-server, function address: 192.168.224.209:30330
> 2020-10-19 13:43:36,332 [warning] warning!, server (0.5.2) and client (unstable) ver dont match


'http://192.168.224.209:30330'

##  Test server

In [24]:
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}'''
fn.invoke('/v2/models/mymodel/infer', my_data)

{'id': '880fcf60-0658-4d78-b769-6d026f762494',
 'model_name': 'mymodel',
 'outputs': [0, 2]}